# EXPERIMENTAL DESIGN FOR CAUSAL EFFECT IDENTIFICATION

We import some useful libraries.

In [94]:
# Import useful libraries
import numpy as np
from ci_test import ci_test
from scipy.io import loadmat
import networkx as nx
from itertools import chain, combinations, permutations
from helpers import *
from collections import deque
from copy import copy
from matplotlib import pyplot as plt
import time


from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We manually create the toy model on which we are going to test the functions implemented in helpers.

In [95]:
# Defining the nodes
nodes = ['z','t','s','w']

# Initializing both the directed and undirected graph (ADMG)
G_b = nx.Graph()
G_d = nx.DiGraph()

# Adding nodes and edges to the graph
for node in nodes:
    G_b.add_node(node)
    G_d.add_node(node)
G_d.add_edge('z','t')
G_d.add_edge('w','t')
G_d.add_edge('t','s')

G_b.add_edge('z','s')
G_b.add_edge('z','t')
G_b.add_edge('t','w')
G_b.add_edge('w','s')
HHull(G_d, G_b, {'s'})

{'s', 't', 'w', 'z'}

We now define the dictionary containing the costs of intervention and then we run the first proposed solution: the brute force Min Cost Intervention Algorithm.

In [96]:
# Defining the costs
costs = {'z':2, 'w':2, 't':3, 's':1}

# Running the algorithm and printing the output
MinCostIntervention({'s'},G_d, G_b,costs)

{'t'}

Since the previous solution scales exponentially in the number of nodes, we now implement and try an heuristic approach which might return a sub-optimal solution.

In [97]:
# Running the heuristic algorithm and printing the output
heuristic_algorithm(G_d, G_b, {'s'}, costs)

{'t'}

As requested in the project description, we run a sanity check to test the correctness of the functions we have implemented.

In [98]:
#Sanity check
G_d_test = G_d.copy()
for node in A:
    parents = list(G_d.predecessors(node))

    # Remove edges from the parents
    for parent in parents:
        G_d_test.remove_edge(parent, node)
        
HHull(G_d_test,G_b, {'s'})

{'s'}

## Graphs generation

In [ ]:
np.random.seed(42)

n_s = np.arange(10,40,3)

print(n_s)

min_cost_time, heuristic_time = [], []

p_directed, p_bidirected = 0.35, 0.25

for n in n_s:
    
    # We create a costs dictionary
    costs = {num:np.random.randint(1,5) for num in range(n)}
    
    # We create the graphs
    G_n_directed = nx.DiGraph()
    G_n_bidirected = nx.Graph()
    
    for node in range(n):
        G_n_directed.add_node(node)
        G_n_bidirected.add_node(node)
        
    for node1 in range(n):
        for node2 in range(n):
            if node1 != node2:
                flag_directed = np.random.random() < p_directed
                flag_bidirected = np.random.random() < p_bidirected
                if flag_directed:
                    G_n_directed.add_edge(node1,node2)
                if flag_bidirected:
                    G_n_bidirected.add_edge(node1,node2)
    
    # We select S
    S = set([np.random.randint(n)])
    
    
    # We save running time of both algos (min cost intervention algorithm)
    start = time.time()
    min_cost_sol = MinCostIntervention( S, G_n_directed, G_n_bidirected, costs)
    end = time.time()
    min_cost_time.append(end - start)
    
    # We save running time of both algos (heuristic algorithm)
    start = time.time()
    min_cost_sol_heuristic = heuristic_algorithm(G_n_directed, G_n_bidirected, S, costs)
    end = time.time()
    heuristic_time.append(end - start)
    
    print('Iteration')
    print(min_cost_sol, min_cost_sol_heuristic)
    

plt.plot(n_s, min_cost_time, '-o', label = 'MinCostIntervention')
plt.plot(n_s, heuristic_time, '-o', label='Heuristic Algorithm')
plt.legend()
    